In [10]:
import numpy as np
from typing import Callable, TypeVar
from collections import Counter, deque
import itertools
import regex as re
from intervaltree import Interval, IntervalTree


np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()]
}

def search(nodes, start, get_neighbors, end_condition=lambda _, __: False, dfs=True):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft() if dfs else q.pop()
        if end_condition(current, distance):
            return visited, current
        if current in visited:
            continue
        for node in get_neighbors(current, distance):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

In [7]:

def day1():
    loc1, loc2 = zip(*data(1, processors['int_list']))
    part1 = sum(abs(x[0]-x[1]) for x in zip(sorted(loc1), sorted(loc2)))
    counts = Counter(loc2)
    part2 = sum(x*counts[x] for x in loc1)
    return part1, part2

day1()

(1941353, 22539317)

In [80]:
def day2():
    def check_safe(report):
        ascending = sorted(report)
        diffs = np.diff(ascending)
        return max(diffs) <= 3 and min(diffs) >= 1 and (
            report == ascending or
            report == list(reversed(ascending))
        )

    def check_safe_damp(report):
        if check_safe(report):
            return 1, 1
        for damped in itertools.combinations(report, len(report)-1):
            if check_safe(list(damped)):
                return 0, 1
        return 0, 0

    reports = data(2, processors['int_list'])
    safe = np.array((0,0))
    for report in reports:
        safe += check_safe_damp(report)
    return safe

day2()

array([356, 413])

In [76]:
def day3():
    def mul_strings(s):
        x, y = s.split(',')
        return int(x)*int(y)

    instructions = ''.join(data(3))
    matches = list(re.finditer(r'mul\((\d+,\d+)\)', instructions))
    conds = list(re.finditer(r"don't\(\).+?do\(\)", instructions))
    donts = IntervalTree([Interval(*cond.span()) for cond in conds])
    result = sum([mul_strings(mul[1]) * (1 if not donts[mul.span()[0]] else 1j) for mul in matches])
    return int(result.real+result.imag), int(result.real)

day3()

(182780583, 90772405)

In [ ]:
def day4():
    grid = np.array(data(0, lambda x: np.array(list(x))))
    target = 'XMAS'
    ymax, xmax = grid.shape
    print(grid)

    count = 0

    def get_neighbors(current, distance):
        col, row = int(current.real), int(current.imag)
        target_letter = target[distance]
        if grid[row, col] != target_letter:
            return
        if distance == len(target)-1:
            total.add(current)
            return

        for v in [1, -1, 1j, -1j, 1+1j, 1-1j, -1+1j, -1-1j]:
            new = current + v
            x, y = int(new.real), int(new.imag)
            if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                continue
            yield new

    for j in range(ymax):
        for i in range(xmax):
            total = set()
            search(grid, i+1j*j, get_neighbors)
            count += len(total)
            print(i, j, count, total)
            # print(i, j, i+1j*j, len(total), total)

    return count

day4()

0 0 0 set()
1 0 0 set()
2 0 0 set()
3 0 0 set()
4 0 3 {(7+3j), (1+1j), (5+3j)}
5 0 7 {(8+0j), (8+1j), (5+3j), (7+3j)}
6 0 7 set()
7 0 7 set()
8 0 7 set()
9 0 7 set()
0 1 7 set()
1 1 7 set()
2 1 7 set()
3 1 7 set()
4 1 10 {(7+3j), (1+1j), (3+4j)}
5 1 10 set()
6 1 10 set()
7 1 10 set()
8 1 10 set()
9 1 10 set()
0 2 10 set()
1 2 10 set()
2 2 11 {(5+3j)}
3 2 11 set()
4 2 14 {(7+3j), (1+3j), (1+1j)}
5 2 14 set()
6 2 14 set()
7 2 14 set()
8 2 14 set()
9 2 14 set()
0 3 14 set()
1 3 14 set()
2 3 14 set()
3 3 14 set()
4 3 14 set()
5 3 14 set()
6 3 14 set()
7 3 14 set()
8 3 14 set()
9 3 19 {(8+0j), (6+1j), (6+6j), (8+6j), (9+6j)}
0 4 23 {(3+2j), (1+1j), (2+6j), (3+4j)}
1 4 23 set()
2 4 23 set()
3 4 23 set()
4 4 23 set()
5 4 23 set()
6 4 27 {(6+1j), (3+2j), (8+1j), (3+4j)}
7 4 27 set()
8 4 27 set()
9 4 27 set()
0 5 29 {(3+2j), (3+4j)}
1 5 31 {(3+2j), (3+4j)}
2 5 31 set()
3 5 31 set()
4 5 31 set()
5 5 32 {(3+2j)}
6 5 34 {(3+2j), (3+4j)}
7 5 34 set()
8 5 34 set()
9 5 34 set()
0 6 34 set()
1 6 34 se

58